----
----
----
----

## Weight Standardization (WS) neden işe yarıyor?

Weight Standardization’da (WS) **optimizer gerçek ağırlığı (`w`) öğrenir**,  
ama **forward sırasında kullanılan ağırlık her zaman standardize edilmiş halidir (`ŵ`)**.

Bu ne demek?

- Model **her zaman aynı ölçekli (mean≈0, std≈1) ağırlıkla** hesap yapar.
- Ağırlıkların büyüyerek veya küçülerek “hile yapması” engellenir.
- Öğrenme **ölçek üzerinden değil, desen/pattern üzerinden** olur.

Önemli nokta:
- Loss, **standardize edilmiş ağırlıkla yapılan forward** sonucuna göre hesaplanır.
- Gradient bu loss’tan gelir ve **gerçek ağırlığı** günceller.
- Yani model, **“standardize edilmiş halim iyi sonuç versin”** diye öğrenmek zorunda kalır.

Sonuç:
- Katmanlar arası ölçek kayması azalır
- Gradient patlaması ihtimali düşer
- Özellikle **GroupNorm + küçük batch** senaryosunda eğitim daha stabil olur

Özetle:
> **WS, ağırlığı kalıcı olarak değiştirmez;  
> ama modeli, ağırlığı her zaman kontrollü bir formda kullanmaya zorlar.**





----
----
----
----


# Weight Standardization (WS) — Baştan Sona (En Temelden En İleriye)
## Tek GPU / Detection-Segmentation Odaklı, PyTorch Uygulamalı

Bu notebook’ta **Weight Standardization** konusunu **sıfırdan** başlayarak, sağlam ve net şekilde kuruyoruz:

- WS nedir? Ne değildir?
- Neden “weight” standardize edilir?
- Matematik: Conv ağırlıklarını kanal bazında standardize etmek
- BN / GN / LN ile ilişkisi (özellikle **GN + WS** kombinasyonu)
- Nerelerde kullanılır? Nerelerde gereksiz?
- PyTorch’ta 3 farklı uygulama:
  1) **WSConv2d** (drop-in Conv2d)
  2) **Parametrization** ile WS (daha temiz)
  3) Var olan modeli otomatik dönüştürme
- Mini demo: ölçek stabilitesi sezgisi
- Pratik checklist (detection/segmentation)

> Önemli: WS, **aktivasyonu değil ağırlığı** normalize eder. Bu yüzden GN/LN gibi aktivasyon normlarıyla birlikte en iyi sonucu verir.



## 1) En temelden: “Standardizasyon” ne demek?

Bir vektörü/diziyi standardize etmek:
- ortalamasını 0’a yaklaştırmak
- standart sapmasını 1’e yaklaştırmak

Formül (eleman bazında):
\[
z = \frac{x - \mu}{\sigma + \epsilon}
\]

Buradaki amaç:
- sayısal ölçekleri kontrol etmek
- optimizasyonu daha stabil yapmak
- “bir katmanın ağırlıkları çok büyüdü, diğeri çok küçük kaldı” gibi dengesizlikleri azaltmak



## 2) Weight Standardization (WS) nedir?

**WS = Convolution (veya Linear) ağırlıklarını** forward sırasında standardize etmek demektir.

Yani:
- Her forward’da weight tensorünü al
- Her output kanal için (out_channel) weight mean’ini çıkar
- std’ye böl
- Sonra standart Conv’u bu “standardize weight” ile yap

### WS ne değildir?
- BatchNorm değildir (aktivasyon istatistiği yok)
- SyncBN değildir (GPU’lar arası istatistik yok)
- “Batch küçükse BN’i düzeltir” iddiası **tek başına** doğru değil

WS’nin ana rolü: **weight ölçeğini stabilize etmek**.



## 3) WS neden işe yarar? (Sezgi)

Convolution’da çıktı ölçeğini en çok belirleyen şeylerden biri **weight ölçeği**dir.
Eğer bazı katmanlarda weight dağılımı “kayarsa” (mean büyür, std artar):
- aktivasyon ölçeği zıplar
- gradient ölçeği zıplar
- eğitim kararsızlaşır

WS şu etkiyi verir:
- Her output kanalın weight’ini “benzer ölçek”e çeker
- Böylece katmanlar arası ölçek drift’i azalır
- Özellikle **GN/LN** gibi batch bağımsız normlarla birleşince güzel stabilite sağlar

> Detection/segmentation’da küçük batch olduğu için GN yaygın; GN ile WS kombinasyonu bu yüzden popüler.



## 4) Matematik: Conv2d’de WS nasıl uygulanır?

Conv2d weight şekli:
- \(W \in \mathbb{R}^{O \times I \times kH \times kW}\)
  - O: out_channels
  - I: in_channels

WS (klasik) per-output-channel yapılır:

1) Her output kanal için mean:
\[
\mu_o = \frac{1}{I kH kW} \sum W_o
\]

2) Her output kanal için std:
\[
\sigma_o = \sqrt{\frac{1}{I kH kW} \sum (W_o - \mu_o)^2 + \epsilon}
\]

3) Standardize:
\[
\hat{W}_o = \frac{W_o - \mu_o}{\sigma_o}
\]

4) Conv’u \(\hat{W}\) ile yap:
\[
y = x * \hat{W}
\]

Bu kadar. İstatistik **batch’ten değil**, weight’in kendisinden geliyor.



## 5) WS hangi normlarla iyi gider?

### 5.1. WS + GroupNorm (en yaygın pratik)
- GN aktivasyonu normalize eder (sample içi)
- WS ağırlığı normalize eder (channel başı)
- Küçük batch’te çok stabil

### 5.2. WS + LayerNorm
- Transformer tabanlı conv-lar / hibritlerde kullanılabilir
- Ama klasik CNN detection tarafında GN daha yaygın

### 5.3. WS + BatchNorm
- BN zaten aktivasyon ölçeğini batch üzerinden ayarlar
- WS bazen yardımcı olur ama çoğu zaman “olmazsa olmaz” değil
- Küçük batch’te BN zaten sorunlu; GN tercih edilir



## 6) Nerelerde kullanılır? Nerelerde gereksiz?

### Kullan ✅ (çok olası)
- Detection / segmentation + küçük batch
- GN kullanan CNN backbone/head
- Deep conv ağları: ölçek drift’ine hassas
- Training stabilitesi (loss zıplaması, grad patlaması) yaşanıyorsa

### Kullanma / dikkat ⚠️
- Zaten BN + büyük batch ile çok stabil gidiyorsan: etkisi sınırlı olabilir
- Çok hızlı inference istiyorsan: WS forward’da ekstra hesap getirir (genelde küçük ama var)
- Quantization / fused conv-bn optimizasyonlarında ek iş çıkarabilir (pipeline’a bağlı)

> WS genelde “stabilite için küçük bir masraf”tır.



## 7) PyTorch Uygulama 1: WSConv2d (en pratik)

Aşağıdaki sınıf, `nn.Conv2d` gibi davranır ama forward’da weight’i standardize eder.
Drop-in replacement: Conv2d yerine WSConv2d koyarsın.


**w.flatten(1).std(dim=1) her out_channel filtre için std hesaplar. Sonra w / w_std yaparak her filtrenin ölçeğini 1’e çekersin; bu da katmanlar arası aktivasyon/gradient ölçeğinin zıplamasını azaltır. Conv’un diğer parametreleri aynı kalır; sadece forward’da kullanılan weight standardize edilir.**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WSConv2d(nn.Conv2d):
    """Weight Standardization uygulanmış Conv2d"""
    def forward(self, x):
        # w.shape = (O, I, kH, kW)
            # O: out_channels (filtre sayısı)
            # I: in_channels
            # kH,kW: kernel boyutu

        w = self.weight
        # Her out_channel için mean/std
        w_mean = w.mean(dim=(1,2,3), keepdim=True)
        w = w - w_mean

        # flatten(1) demek: 0. boyutu (O) koru, geri kalan her şeyi düzleştir.
        w_std = w.flatten(1).std(dim=1, keepdim=True).view(-1,1,1,1) + 1e-5 # O=64 ise → 64 tane std çıkar (her filtreye 1 std).
        # Şu anda w_std şekli (O,1). Ama biz bunu w ile bölmek istiyoruz :: w şekli (O, I, kH, kW) :: Broadcast düzgün olsun diye w_std’yi (O,1,1,1) yapıyoruz.

        w = w / w_std

        # Forward sırasında kullanılan weight (w) standardize edilmiş oluyor.
        # Bias, stride, padding, dilation, groups aynı kalıyor.
        # Modelin parametresi kalıcı olarak değişmiyor
        return F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        
# quick test
conv = WSConv2d(3, 16, 3, padding=1, bias=False)
x = torch.randn(2,3,64,64)
y = conv(x)
y.shape

torch.Size([2, 16, 64, 64])


## 8) WS + GN blok (Detection/Segmentation için gerçekçi)

Bu blok: `WSConv2d -> GroupNorm -> SiLU`
Bu tarz bloklar U-Net, FPN head, backbone gibi yerlerde rahatça kullanılır.


In [3]:
class WSGNBlock(nn.Module):
    def __init__(self, in_ch, out_ch, groups=32, k=3, s=1, p=1):
        super().__init__()
        self.conv = WSConv2d(in_ch, out_ch, kernel_size=k, stride=s, padding=p, bias=False)
        g = min(groups, out_ch)
        while out_ch % g != 0 and g > 1:
            g -= 1
        self.gn = nn.GroupNorm(num_groups=g, num_channels=out_ch)
        self.act = nn.SiLU(inplace=True)

    def forward(self, x):
        return self.act(self.gn(self.conv(x)))

blk = WSGNBlock(3, 64, groups=32)
blk(torch.randn(2,3,128,128)).shape

torch.Size([2, 64, 128, 128])


## 9) PyTorch Uygulama 2: Parametrization ile WS (daha temiz yaklaşım)

PyTorch `torch.nn.utils.parametrize` ile bir parametreye “dönüşüm” ekleyebilirsin.
Avantaj:
- Modül kodunu çok bozmadan weight üzerinde WS uygularsın
- Daha modüler olur

Aşağıdaki örnek, Conv2d weight’ine WS parametrizasyonu uygular.


#### iki katman var:

* Orijinal (öğrenilen) parametre → optimizer bunun üstünden günceller

* Görünen parametre (forward’da kullanılan) → orijinalden her seferinde hesaplanır

##### Şunu yaptığımızda:
```python 
parametrize.register_parametrization(conv, "weight", WeightStandardization())
```
##### PyTorch şunu yapar:

* conv.weight artık bir Tensor parametresi değil, hesaplanan bir property gibi davranır.

* Asıl öğrenilen parametre şuraya taşınır: **conv.parametrizations.weight.original**

Biz conv.weight dediğimizde, PyTorch otomatik olarak: **WeightStandardization()(conv.parametrizations.weight.original)** döndürür.

##### **Yani forward’da kullanılan weight = standardize edilmiş ağırlık.**

In [4]:
import torch.nn.utils.parametrize as parametrize

class WeightStandardization(nn.Module):
    def __init__(self, eps=1e-5):
        super().__init__()
        self.eps = eps

    def forward(self, w):
        w_mean = w.mean(dim=(1,2,3), keepdim=True)
        w = w - w_mean
        w_std = w.flatten(1).std(dim=1, keepdim=True).view(-1,1,1,1) + self.eps
        return w / w_std

conv2 = nn.Conv2d(3, 16, 3, padding=1, bias=False)
parametrize.register_parametrization(conv2, "weight", WeightStandardization(eps=1e-5))

x = torch.randn(2,3,64,64)
y2 = conv2(x)
y2.shape

torch.Size([2, 16, 64, 64])

## Bunu metafora vuralım

#### **Parametrize = Filtre takmak 🧃**

Şimdi düşünelim:

* Elimizde orijinal meyve suyu var → bu gerçek weight

Biz içmeden önce:

* süzgeçten geçiriyorsuz

* tadını dengeliyorsuz

Ama:

* Meyve suyunu döküp yenisini koymuyoruz

* Şişeyi değiştirmiyoruz

* Sadece içerken süzüyoruz

❌ Parametreyi başka parametreyle değiştirmiyor

❌ Kalıcı kopya koymuyor

✅ Aynı parametreyi, kullanmadan önce makyajlıyor


### Parametrization notu
Bu yöntemde:
- `conv2.weight` artık “parametrized” bir görünüm olur
- Orijinal parametre `conv2.parametrizations.weight.original` içinde durur



## 10) PyTorch Uygulama 3: Model içindeki Conv2d’leri otomatik WS’e dönüştürme

Elinde büyük bir model var diyelim (backbone + head). Hepsini tek tek elle değiştirmek eziyet.
Aşağıdaki fonksiyon, model içinde gezip **Conv2d → WSConv2d** yapar.


In [5]:
def convert_conv_to_wsconv(module: nn.Module):
    for name, child in module.named_children():
        if isinstance(child, nn.Conv2d) and not isinstance(child, WSConv2d):
            ws = WSConv2d(
                in_channels=child.in_channels,
                out_channels=child.out_channels,
                kernel_size=child.kernel_size,
                stride=child.stride,
                padding=child.padding,
                dilation=child.dilation,
                groups=child.groups,
                bias=(child.bias is not None),
                padding_mode=child.padding_mode,
            )
            with torch.no_grad():
                ws.weight.copy_(child.weight)
                if child.bias is not None:
                    ws.bias.copy_(child.bias)
            setattr(module, name, ws)
        else:
            convert_conv_to_wsconv(child)
    return module

# demo
class TinyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(3, 16, 3, padding=1, bias=False)
        self.mid = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1, bias=False),
        )
    def forward(self, x): return self.mid(self.stem(x))

m = TinyNet()
print("Önce:", m)
m = convert_conv_to_wsconv(m)
print("Sonra:", m)

Önce: TinyNet(
  (stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (mid): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
)
Sonra: TinyNet(
  (stem): WSConv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (mid): Sequential(
    (0): WSConv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): WSConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
)



## 11) Mini demo: WS’in “ölçek stabilitesi” sezgisi

Bu demo eğitim performansını ispatlamaz; ama WS’in weight dağılımını kontrol ettiğini görürsün.

- Normal Conv: weight’in channel mean/std’si serbest
- WSConv: forward’da her out_channel için mean ~0, std ~1’e zorlanır


In [6]:
torch.manual_seed(0)
conv_plain = nn.Conv2d(3, 8, 3, padding=1, bias=False)
conv_ws = WSConv2d(3, 8, 3, padding=1, bias=False)
with torch.no_grad():
    conv_ws.weight.copy_(conv_plain.weight)

w = conv_plain.weight.detach()
w_mean = w.mean(dim=(1,2,3))
w_std  = w.flatten(1).std(dim=1)

# WS sonrası efektif weight'i hesaplayalım (forward içindeki gibi)
w2 = conv_ws.weight.detach()
w2 = w2 - w2.mean(dim=(1,2,3), keepdim=True)
w2 = w2 / (w2.flatten(1).std(dim=1, keepdim=True).view(-1,1,1,1) + 1e-5)
w2_mean = w2.mean(dim=(1,2,3))
w2_std  = w2.flatten(1).std(dim=1)

print("Plain weight mean (ilk 5):", w_mean[:5])
print("Plain weight std  (ilk 5):", w_std[:5])
print("WS    weight mean (ilk 5):", w2_mean[:5])
print("WS    weight std  (ilk 5):", w2_std[:5])

Plain weight mean (ilk 5): tensor([-0.0016, -0.0173,  0.0059, -0.0146, -0.0172])
Plain weight std  (ilk 5): tensor([0.1036, 0.1192, 0.1182, 0.1214, 0.0989])
WS    weight mean (ilk 5): tensor([ 2.6491e-08,  4.4152e-08, -1.3245e-08,  1.3245e-08,  8.8303e-09])
WS    weight std  (ilk 5): tensor([0.9999, 0.9999, 0.9999, 0.9999, 0.9999])



## 12) Detection/Segmentation’da WS’yi nereye koyarım?

Pratik yerleşim:
- Backbone conv blokları
- FPN / decoder conv blokları
- Head conv blokları (box/class/mask)

En yaygın pattern:
- `WSConv2d -> GroupNorm -> Activation`

Özellikle:
- batch=1–4
- GN kullanıyorsun
- loss dalgalanması / grad patlaması görüyorsun
→ WS eklemek mantıklı.



## 13) Sık hatalar ve tuzaklar

- **WS’yi BN ile “küçük batch çözümü” sanmak:** Tek başına BN sorununu çözmez.
- **GN gruplarını yanlış seçmek:** `out_channels % num_groups == 0` olmalı.
- **Bias kullanımı:** WSConv2d’de bias kullanılabilir ama birçok backbone’da bias=False tercih edilir (GN zaten shift yapıyor).
- **Inference optimizasyonu:** Bazı deployment/fusion adımlarında ekstra işlem olabilir; training’de genelde problem değil.



## 14) Hızlı checklist

- [ ] Batch küçük mü? (1–4–8)  
- [ ] Normalization GN mi? (evet olmalı)  
- [ ] Training dalgalı mı?  
- [ ] WS + GN block denedin mi?  
- [ ] LR/warmup/AMP ayarları yerinde mi?  

Önerilen başlangıç:
- GN (groups=32, kanal küçükse 16/8)
- WSConv2d + GN + SiLU
- AMP açık
- LR biraz daha düşük + warmup
